In [1]:
from fastai import *
from fastai.basics import *

In [21]:
path = Path('../Data')
path.ls()

[PosixPath('PKLot'),
 PosixPath('prepare_dataset.ipynb'),
 PosixPath('data'),
 PosixPath('.ipynb_checkpoints'),
 PosixPath('darknet')]

In [22]:
img_names = ItemList.from_folder(path/'PKLot'/'PKLot', extensions='.jpg')
label_names = ItemList.from_folder(path/'PKLot'/'PKLot', extensions='.xml')

In [23]:
img_names[0]

PosixPath('PKLot/PKLot/UFPR05/Sunny/2013-04-10/2013-04-10_10_30_05.jpg')

In [24]:
label_names[0]

PosixPath('PKLot/PKLot/UFPR05/Sunny/2013-04-10/2013-04-10_07_30_01.xml')

In [25]:
# Get all image paths as a single python list
imgs = []
labels = []

for img in img_names:
    img = str(img)
    imgs.append(img)

for label in label_names:
    label = str(label)
    labels.append(label)
    
imgs.sort()
labels.sort()
# This deletion is different for different runs
# del imgs[4135]

# Check imgs and labels for correctness
for index, (img, label) in enumerate(zip(imgs, labels)):
    img_name = img.split('.')[0]
    lbl_name = label.split('.')[0]
#     if img_name != lbl_name:
#         print(index)
#         print(imgs[index])
#         print(labels[index])
#         break
    if img_name != lbl_name:
        print(f'img = {img_name} & label = {lbl_name}')

In [37]:
# Move all the images to 'data' dir
import shutil
from lxml import etree
from PIL import Image
dest = path/'data'
counter = 0

for img, label in zip(imgs, labels):
    # Handle the image
    img_name = str(dest/f'{counter:08d}.jpg')
    shutil.copy(img, img_name)
    
    # Handle the label
    # For yolo we need to have a txt with same name as img with the labels
    img = Image.open(img_name)
    img_w, img_h = img.size
    
    lbl_name = str(dest/f'{counter:08d}.txt')
    f = open(lbl_name, 'w')
    label_xml = etree.parse(label)
    root = label_xml.getroot()
    
    for child in root:
        try:
            label_id = child.attrib['occupied']
        except:
            continue
        rect = child.find('rotatedRect')
        center = rect.find('center')
        size = rect.find('size')
        # Skipping angle for now
        
        x = int(center.attrib['x']) / img_w
        y = int(center.attrib['y']) / img_h
        w = int(size.attrib['w'])   / img_w
        h = int(size.attrib['h'])   / img_h
        
        label_str = f'{label_id} {x} {y} {w} {h}\n'
        f.write(label_str)
        
        counter += 1
    f.close()

In [38]:
# Create a text file containing all image names
data = ItemList.from_folder(path/'data', extensions='.jpg')
prefix = Path('/home/kushaj/Desktop/Github/Parking Lot')

# Write to text file
f = open(path/'darknet/parking_lot/train.txt', 'w')
for img in data:
    img = str(prefix/img) + '\n'
    f.write(img)
f.close()